In [492]:
import pandas as pd
import numpy as np
from collections import Counter
import random as r
from sklearn.model_selection import train_test_split
dt = pd.read_csv('../data/model3.csv')
# откуда то при пересохранении в csv образуется лишний столбец
dt.drop(dt.columns[[0]], axis = 1, inplace = True)

# балансируем классы и разбиваем на выборки
# в тестовой выборке делаем баланс более близким к реальности, а в обучающей - близкий к равновесному
dt['cat'] = dt['claim'].apply(lambda x: r.choice(list(range(1, 5))) if x==1 else r.choice(list(range(10, 29))))
kbmbad = dt[dt['claim'] == 1].AGR_KBM.mean()
dt['kbm_gain'] = dt['AGR_KBM'].apply(lambda x: (kbmbad-x)*10**2)
trn = dt[dt['cat'].isin([10, 2, 3, 4, 5])].drop('cat', axis = 1)
tst = dt[dt['cat'].isin([1,11, 12, 15, 16, 17])].drop('cat', axis = 1)

X_train = trn.drop(['claim'], axis = 1)
X_test = tst.drop(['claim'], axis = 1)
y_train = trn.claim
y_test = tst.claim

df = pd.read_csv('../data/model1.csv')
df.drop(df.columns[[0]], axis = 1, inplace = True)
kbmbad = df[df['claim'] == 1].AGR_KBM.mean()
df['kbm_gain'] = df['AGR_KBM'].apply(lambda x: (kbmbad-x)*10**2)
X = df.drop(['claim'], axis = 1)
Y = df.claim
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y, test_size=0.2, random_state=42)

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
dtmms = pd.DataFrame(min_max_scaler.fit_transform(df))
dtmms.columns = df.columns
Xmms = dtmms.drop(['claim'], axis = 1)
Ymms = dtmms.claim
X2_train, X2_test, y2_train, y2_test = train_test_split(Xmms, Ymms, test_size=0.2, random_state=42)


print(Counter(dt.claim), Counter(y_train), Counter(y_test))
dtmms.info()

Counter({0: 78203, 1: 3767}) Counter({0: 4185, 1: 2828}) Counter({0: 20485, 1: 939})
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106806 entries, 0 to 106805
Data columns (total 68 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   INS_JUR                                        106806 non-null  float64
 1   OWN_JUR                                        106806 non-null  float64
 2   AGR_KBM                                        106806 non-null  float64
 3   TS_POWER                                       106806 non-null  float64
 4   f1_daysbeforestart                             106806 non-null  float64
 5   f2_AGR_SRC_True                                106806 non-null  float64
 6   f2_AGR_SRC_False                               106806 non-null  float64
 7   f3_AGR_CREATEDBY_True                          106806 non-null  float64
 8   f3_AGR_CREATEDBY_False       

In [493]:
from scipy.optimize import minimize

def mtr(m, x = X_test, y = y_test, limit = 0):
#     print(len(x), len(y))
    pr = m.predict(x)
#     print(len(pr))
    if limit == 0:
        pred = pr
    elif limit == 1:
        def min(limit, pr=pr):
            pr = np.where(pr > limit, 1, 0)
            ras = roc_auc_score(y, pr)
            return -ras
        l = minimize(min, 0.1, method='COBYLA')['x']
        print ('Оптимальный лимит: ', l)
        print('')
        pred = np.where(pr > l, 1, 0)
    else:
        pred = np.where(pr > limit, 1, 0)
    print(confusion_matrix(y, pred))
    print('')
    print(classification_report(y, pred, output_dict=True))
    print('')
    print('ROC-AUC score: ', roc_auc_score(y, pred))

In [495]:
def min(limit):
    pr = models.predict(X1_test)
    pred = np.where(pr > limit, 1, 0)
    ras = roc_auc_score(y1_test, pred)
    return -ras

minimize(min, 0.1, method='COBYLA')['x']

ValueError: in user code:

    C:\Users\photo\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:1544 predict_function  *
        return step_function(self, iterator)
    C:\Users\photo\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:1527 run_step  *
        outputs = model.predict_step(data)
    C:\Users\photo\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:1500 predict_step  *
        return self(x, training=False)
    C:\Users\photo\AppData\Roaming\Python\Python38\site-packages\keras\engine\base_layer.py:989 __call__  *
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\photo\AppData\Roaming\Python\Python38\site-packages\keras\engine\input_spec.py:248 assert_input_compatibility  *
        raise ValueError(

    ValueError: Input 0 of layer sequential_60 is incompatible with the layer: expected axis -1 of input shape to have value 66 but received input with shape (None, 67)


In [496]:
# попробуем решить задачу ансамблем деревьев
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt

modelrf = RandomForestClassifier(max_depth = 16)
modelrf.fit(X_train, y_train)
mtr(modelrf, x = X_test, y = y_test, limit = 1)

Оптимальный лимит:  0.1

[[17009  3476]
 [  619   320]]

{'0': {'precision': 0.964885409575675, 'recall': 0.8303148645350257, 'f1-score': 0.8925563456038622, 'support': 20485}, '1': {'precision': 0.08429926238145416, 'recall': 0.3407880724174654, 'f1-score': 0.1351636747624076, 'support': 939}, 'accuracy': 0.8088592233009708, 'macro avg': {'precision': 0.5245923359785646, 'recall': 0.5855514684762455, 'f1-score': 0.5138600101831349, 'support': 21424}, 'weighted avg': {'precision': 0.9262898908949725, 'recall': 0.8088592233009708, 'f1-score': 0.8593603169481432, 'support': 21424}}

ROC-AUC score:  0.5855514684762454


In [497]:
# попробуем другие модели
from sklearn.ensemble import GradientBoostingClassifier
modelgb = GradientBoostingClassifier()
modelgb.fit(X1_train, y1_train)
mtr(modelgb, x = X1_test, y = y1_test, limit = 0)

[[20216     9]
 [ 1134     3]]

{'0': {'precision': 0.9468852459016394, 'recall': 0.9995550061804697, 'f1-score': 0.9725075165363801, 'support': 20225}, '1': {'precision': 0.25, 'recall': 0.002638522427440633, 'f1-score': 0.005221932114882507, 'support': 1137}, 'accuracy': 0.9464937739911993, 'macro avg': {'precision': 0.5984426229508197, 'recall': 0.5010967643039551, 'f1-score': 0.48886472432563133, 'support': 21362}, 'weighted avg': {'precision': 0.9097932823874476, 'recall': 0.9464937739911993, 'f1-score': 0.9210233994365186, 'support': 21362}}

ROC-AUC score:  0.5010967643039551


In [498]:
from catboost import CatBoostClassifier

modelcb = CatBoostClassifier(
        loss_function='Logloss',
        eval_metric='AUC',
        logging_level='Silent',
        allow_writing_files=False
    )

modelcb.fit(X_train, y_train)
mtr(modelcb, x = X_test, y = y_test, limit = 0)

[[16864  3621]
 [  602   337]]

{'0': {'precision': 0.9655330356120463, 'recall': 0.8232365145228215, 'f1-score': 0.8887249347843271, 'support': 20485}, '1': {'precision': 0.08514401212733703, 'recall': 0.3588924387646432, 'f1-score': 0.13763528691035326, 'support': 939}, 'accuracy': 0.8028846153846154, 'macro avg': {'precision': 0.5253385238696917, 'recall': 0.5910644766437324, 'f1-score': 0.5131801108473402, 'support': 21424}, 'weighted avg': {'precision': 0.9269461567354527, 'recall': 0.8028846153846154, 'f1-score': 0.8558051635299553, 'support': 21424}}

ROC-AUC score:  0.5910644766437324


In [499]:
import xgboost
modelxg = xgboost.XGBClassifier(max_depth = 8, use_label_encoder=False, objective = 'binary:logistic')
modelxg.fit(X2_train, y2_train)
mtr(modelxg, x = X2_test, y = y2_test, limit = 0)

[09:38:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[20170    55]
 [ 1106    31]]

{'0.0': {'precision': 0.948016544463245, 'recall': 0.9972805933250927, 'f1-score': 0.9720247704874582, 'support': 20225}, '1.0': {'precision': 0.36046511627906974, 'recall': 0.027264731750219876, 'f1-score': 0.050695012264922325, 'support': 1137}, 'accuracy': 0.9456511562587773, 'macro avg': {'precision': 0.6542408303711573, 'recall': 0.5122726625376562, 'f1-score': 0.5113598913761902, 'support': 21362}, 'weighted avg': {'precision': 0.9167439120390616, 'recall': 0.9456511562587773, 'f1-score': 0.9229866684792649, 'support': 21362}}

ROC-AUC score:  0.5122726625376562


In [500]:
# попробуем использовать неиросеть на базе Keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import metrics
import keras.backend as K

METRICSS = [
#       metrics.TruePositives(name='tp'),
#       metrics.FalsePositives(name='fp'),
#       metrics.TrueNegatives(name='tn'),
#       metrics.FalseNegatives(name='fn'), 
#       metrics.BinaryAccuracy(name='accuracy'),
#       metrics.Precision(name='precision'),
#       metrics.Recall(name='recall'),
#       metrics.BinaryCrossentropy(name="binary_crossentropy")
      metrics.AUC(name='auc'),
#       metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

models = Sequential()
models.add(Dense(12, input_dim=X1_train.shape[1], activation='relu'))
# models.add(Dense(42, activation='relu'))
models.add(Dense(8, activation='relu'))
models.add(Dense(8, activation='relu'))
# modelmms.add(Dropout(0.9))
models.add(Dense(1, activation='sigmoid')
      
         )
# компилируем модель
models.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICSS)

In [504]:
models.fit(X1_train, y1_train, batch_size=2000, epochs=10, verbose=0)#0.621
# mtr(models)
mtr(models, x = X1_test, y = y1_test, limit = 1)

Оптимальный лимит:  0.053025

[[14290  5935]
 [  549   588]]

{'0': {'precision': 0.9630028977693915, 'recall': 0.7065512978986404, 'f1-score': 0.8150809947524528, 'support': 20225}, '1': {'precision': 0.0901425724359957, 'recall': 0.5171503957783641, 'f1-score': 0.1535248041775457, 'support': 1137}, 'accuracy': 0.6964703679430765, 'macro avg': {'precision': 0.5265727351026936, 'recall': 0.6118508468385022, 'f1-score': 0.4843028994649992, 'support': 21362}, 'weighted avg': {'precision': 0.9165445984573388, 'recall': 0.6964703679430766, 'f1-score': 0.7798694326944213, 'support': 21362}}

ROC-AUC score:  0.6118508468385022


In [386]:
# models.save('../models/keras_model3_v2_0624.pbtxt')

INFO:tensorflow:Assets written to: ../models/keras_model3_v2_0624.pbtxt\assets


In [480]:
res = models.predict(X1_test)
data1_res = X1_test.join(pd.DataFrame(res)).join(pd.DataFrame(y1_test))
data1_res.columns = [                                       'INS_JUR',
                                             'OWN_JUR',
                                            'TS_POWER',
                                  'f1_daysbeforestart',
                                     'f2_AGR_SRC_True',
                                    'f2_AGR_SRC_False',
                               'f3_AGR_CREATEDBY_True',
                              'f3_AGR_CREATEDBY_False',
                                     'f4_TS_TYPE_True',
                                    'f4_TS_TYPE_False',
                                 'f5_TS_CATEGORY_True',
                                'f5_TS_CATEGORY_False',
                                'f6_TS_MARK_NAME_True',
                               'f6_TS_MARK_NAME_False',
                               'f7_TS_MODEL_NAME_True',
                              'f7_TS_MODEL_NAME_False',
                                 'f8_car_age_on_start',
                                     'f9_OWN_REG_True',
                                    'f9_OWN_REG_False',
                                    'f10_INS_REG_True',
                                   'f10_INS_REG_False',
                                 'f11_INS_DOCSER_True',
                                'f11_INS_DOCSER_False',
                                 'f12_OWN_DOCSER_True',
                                'f12_OWN_DOCSER_False',
                                      'f13_own_is_ins',
                                 'f14_is_prolongation',
                                   'f15_drivers_count',
                                     'f17_drv_age_min',
                                     'f18_drv_age_max',
                                   'f19_drv_skill_min',
                                   'f20_drv_skill_max',
                                         'f16_kbm_min',
                                       'f21_oneperson',
                                        'f22_agr_days',
                                           'OWN_SEX_Ж',
                                           'OWN_SEX_М',
                                           'INS_SEX_Ж',
                                           'INS_SEX_М',
                                    'TS_TYPE_Автобусы',
                         'TS_TYPE_Грузовые автомобили',
         'TS_TYPE_Колесные тракторы и дорожные машины',
                         'TS_TYPE_Легковые автомобили',
                     'TS_TYPE_Мотоциклы и мотороллеры',
                                 'TS_TYPE_Отсутствует',
                                     'TS_TYPE_Трамваи',
                                 'TS_TYPE_Троллейбусы',
                           'TS_CATEGORY_Категория "A"',
                          'TS_CATEGORY_Категория "A1"',
                           'TS_CATEGORY_Категория "B"',
                           'TS_CATEGORY_Категория "C"',
                           'TS_CATEGORY_Категория "D"',
                           'TS_CATEGORY_Категория "E"',
                          'TS_CATEGORY_Категория "Tb"',
                          'TS_CATEGORY_Категория "Tm"',
               'TS_USE_NAME_Дорожные и специальные ТС',
                              'TS_USE_NAME_Инкассация',
                                  'TS_USE_NAME_Личная',
                'TS_USE_NAME_Перевозка опасных грузов',
         'TS_USE_NAME_Прокат или краткосрочная аренда',
                                  'TS_USE_NAME_Прочее',
       'TS_USE_NAME_Регулярные пассажирские перевозки',
                 'TS_USE_NAME_Такси, Маршрутное такси',
                            'TS_USE_NAME_Учебная езда',
        'TS_USE_NAME_Экстренные и коммунальные службы',
                                            'kbm_gain',
                                                'res',
                                               'claim']
data1_res['res'] = data1_res['res'].apply(lambda x: 1 if x > 0.03 else 0)
data1_res.head()

,INS_JUR,OWN_JUR,TS_POWER,f1_daysbeforestart,f2_AGR_SRC_True,f2_AGR_SRC_False,f3_AGR_CREATEDBY_True,f3_AGR_CREATEDBY_False,f4_TS_TYPE_True,f4_TS_TYPE_False,...,TS_USE_NAME_Перевозка опасных грузов,TS_USE_NAME_Прокат или краткосрочная аренда,TS_USE_NAME_Прочее,TS_USE_NAME_Регулярные пассажирские перевозки,"TS_USE_NAME_Такси, Маршрутное такси",TS_USE_NAME_Учебная езда,TS_USE_NAME_Экстренные и коммунальные службы,kbm_gain,res,claim
19669,0,0,105.0,8,0.346128,0.406010,0.056263,0.107219,0.060063,0.345434,...,0,0,0,0,0,0,0,41.771568,1,0
86720,0,0,76.7,1,0.522074,0.011886,0.065539,0.101480,0.060063,0.345434,...,0,0,0,0,0,0,0,6.771568,0,0
12772,0,0,150.0,0,0.346128,0.406010,0.046392,0.100229,0.060063,0.345434,...,0,0,0,0,0,0,0,1.771568,1,0
95687,0,0,123.0,13,0.000000,1.000000,0.056086,0.137232,0.060063,0.345434,...,0,0,0,0,0,0,0,-8.228432,0,0
77756,1,1,149.0,8,0.522074,0.011886,0.071267,0.106326,0.060063,0.345434,...,0,0,1,0,0,0,0,11.771568,0,0


In [479]:
data1_res.columns

Index([                                      'INS_JUR',
                                             'OWN_JUR',
                                            'TS_POWER',
                                  'f1_daysbeforestart',
                                     'f2_AGR_SRC_True',
                                    'f2_AGR_SRC_False',
                               'f3_AGR_CREATEDBY_True',
                              'f3_AGR_CREATEDBY_False',
                                     'f4_TS_TYPE_True',
                                    'f4_TS_TYPE_False',
                                 'f5_TS_CATEGORY_True',
                                'f5_TS_CATEGORY_False',
                                'f6_TS_MARK_NAME_True',
                               'f6_TS_MARK_NAME_False',
                               'f7_TS_MODEL_NAME_True',
                              'f7_TS_MODEL_NAME_False',
                                 'f8_car_age_on_start',
                                     'f9_OWN_REG

In [482]:
data1_res[[#'AGR_KBM', 
            'f15_drivers_count', 
            'f17_drv_age_min',
            'f18_drv_age_max',
            'f19_drv_skill_min',
            'f20_drv_skill_max',
            'kbm_gain',
            'f16_kbm_min', 'res', 'claim']].groupby(['res']).mean()

,f15_drivers_count,f17_drv_age_min,f18_drv_age_max,f19_drv_skill_min,f20_drv_skill_max,kbm_gain,f16_kbm_min,claim
res,,,,,,,,
0,1.420746,37.773443,45.96821,19.589724,19.935415,8.454069,0.756081,0.052997
1,1.411807,37.716536,49.02300,25.736368,22.920743,7.632033,0.739758,0.054741


In [483]:
data1_res[[#'AGR_KBM', 
            'f15_drivers_count', 
            'f17_drv_age_min',
            'f18_drv_age_max',
            'f19_drv_skill_min',
            'f20_drv_skill_max',
            'kbm_gain',
            'f16_kbm_min', 'res', 'claim']].groupby(['claim']).mean()

,f15_drivers_count,f17_drv_age_min,f18_drv_age_max,f19_drv_skill_min,f20_drv_skill_max,kbm_gain,f16_kbm_min,res
claim,,,,,,,,
0,1.4311,37.848118,46.434345,20.256866,20.380351,8.740765,0.751567,0.130630
1,1.2146,36.305231,45.185943,22.832405,19.359469,1.333573,0.796249,0.134565
